<a href="https://colab.research.google.com/github/kwazinhlaka/2022-pydata-global-seaborn/blob/main/turtle_sim_experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🐢 Turtle Sim Experiment

Experimenting in `python` with `mujoco` to make virtual `turtle-bots` that can be played with by anyone with access to a browser and internet connection. The goal was to be able to write code in-browser and then see a rendered video of your robot doing the tasks.

Presented at PyconZA 2023 by :David Campey in the session [turtle-bots: simulated beginnings](https://za.pycon.org/talks/41-turtle-bots-simulated-beginnings/).

This experiment uses [**MuJoCo** physics](https://github.com/google-deepmind/mujoco#readme), using their excellent native Python bindings.

Note: **A Colab runtime with GPU acceleration is required.** If you're using a CPU-only runtime, you can switch using the menu "Runtime > Change runtime type".

### Sources & Copyright

This experiment started based on the excellent introductory [Google DeepMind mujoco tutorial](https://github.com/google-deepmind/mujoco/blob/main/python/tutorial.ipynb) tutorial, start there if you want a deeper understanding of using mujoco in python <a href="https://colab.research.google.com/github/google-deepmind/mujoco/blob/main/python/tutorial.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" width="140" align="center"/></a>.

The differential drive is based on

> <p><small><small>Portions copyright 2022 DeepMind Technologies Limited, 2023 Coder:LevelUp NPC.</small></p>
> <p><small><small>Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at <a href="http://www.apache.org/licenses/LICENSE-2.0">http://www.apache.org/licenses/LICENSE-2.0</a>.</small></small></p>
> <p><small><small>Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.</small></small></p>

# Install MuJoCo

In [ ]:
!pip install mujoco


#@title Import packages for plotting and creating graphics
import time
import itertools
import numpy as np
from typing import Callable, NamedTuple, Optional, Union, List

# Graphics and plotting.
print('Installing mediapy:')
!command -v ffmpeg >/dev/null || (apt update && apt install -y ffmpeg)
!pip install -q mediapy
import mediapy as media
import matplotlib.pyplot as plt

# More legible printing from numpy.
np.set_printoptions(precision=3, suppress=True, linewidth=100)

In [ ]:
#@title Check if installation was successful

from google.colab import files

import distutils.util
import os
import subprocess
if subprocess.run('nvidia-smi').returncode:
  raise RuntimeError(
      'Cannot communicate with GPU. '
      'Make sure you are using a GPU Colab runtime. '
      'Go to the Runtime menu and select Choose runtime type.')

# Add an ICD config so that glvnd can pick up the Nvidia EGL driver.
# This is usually installed as part of an Nvidia driver package, but the Colab
# kernel doesn't install its driver via APT, and as a result the ICD is missing.
# (https://github.com/NVIDIA/libglvnd/blob/master/src/EGL/icd_enumeration.md)
NVIDIA_ICD_CONFIG_PATH = '/usr/share/glvnd/egl_vendor.d/10_nvidia.json'
if not os.path.exists(NVIDIA_ICD_CONFIG_PATH):
  with open(NVIDIA_ICD_CONFIG_PATH, 'w') as f:
    f.write("""{
    "file_format_version" : "1.0.0",
    "ICD" : {
        "library_path" : "libEGL_nvidia.so.0"
    }
}
""")

# Configure MuJoCo to use the EGL rendering backend (requires GPU)
print('Setting environment variable to use GPU rendering:')
%env MUJOCO_GL=egl

try:
  print('Checking that the installation succeeded:')
  import mujoco
  mujoco.MjModel.from_xml_string('<mujoco/>')
except Exception as e:
  raise e from RuntimeError(
      'Something went wrong during installation. Check the shell output above '
      'for more information.\n'
      'If using a hosted Colab runtime, make sure you enable GPU acceleration '
      'by going to the Runtime menu and selecting "Choose runtime type".')

print('Installation successful.')

# MuJoCo basics

We begin by defining and creating a simple model, a first approximation of a turtle:

In [ ]:
xml = """
<mujoco>
  <worldbody>
    <geom name="body" type="ellipsoid" size=".2 .1 .1" rgba="0 1 1 1"/>
    <geom name="shell" pos=".2 .2 .2" size=".1" rgba="0 1 0 1"/>
  </worldbody>
</mujoco>
"""
model = mujoco.MjModel.from_xml_string(xml)

The `xml` string is written in MuJoCo's [MJCF](http://www.mujoco.org/book/modeling.html), which is an [XML](https://en.wikipedia.org/wiki/XML#Key_terminology)-based modeling language.

MuJoCo is first and foremost a physics sim, so we have here a spherical approximation of a turtle, but add an ellipsoid body so the head can poke out and give it a "front".

Calling the named accessor without specifying a property will tell us what all the valid properties are:

In [ ]:
model.geom('shell')

And it lets us do things in pythonic ways, lik the `id` and `name` attributes are useful in Python comprehensions:

In [ ]:
[model.geom(i).name for i in range(model.ngeom)]

# Let's see the turtles!!!


In order to render we'll need to instantiate a `Renderer` object and call its `render` method.

We'll also reload our model to make the colab's sections independent.

In [ ]:
xml = """
<mujoco>
  <worldbody>
    <geom name="body" type="ellipsoid" size=".2 .1 .1" rgba="0 1 1 1"/>
    <geom name="shell" pos=".2 .2 .2" size=".1" rgba="0 1 0 1"/>
  </worldbody>
</mujoco>
"""
# Make model and data
model = mujoco.MjModel.from_xml_string(xml)
data = mujoco.MjData(model)

# Make renderer, update scene, render and show the pixels
renderer = mujoco.Renderer(model)
mujoco.mj_forward(model, data)
renderer.update_scene(data)

media.show_image(renderer.render())

This worked, but this image is a bit dark. Also, it seems the shell isn't quite on the body.

Let's fix the shell, add a light and re-render.

In [ ]:
xml = """
<mujoco>
  <worldbody>
    <light name="top" pos="0 0 1"/>
    <geom name="body" type="ellipsoid" size=".2 .1 .1" rgba="0 1 1 1"/>
    <geom name="shell" pos="-0.06 .0 .0" size=".18" rgba="0 1 0 1"/>
  </worldbody>
</mujoco>
"""
model = mujoco.MjModel.from_xml_string(xml)
data = mujoco.MjData(model)
renderer = mujoco.Renderer(model)

mujoco.mj_forward(model, data)
renderer.update_scene(data)

media.show_image(renderer.render())

Much better!

> "It's so cute, now we need to give it fins..." -- Lily

> "Maybe later" :) -- Dad



# Simulation

Now let's simulate and make a video. We'll use MuJoCo's main high level function `mj_step`, which steps the state $x_{t+h} = f(x_t)$.


In [ ]:
duration = 3.8  # (seconds)
framerate = 60  # (Hz)

# Simulate and display video.
frames = []
mujoco.mj_resetData(model, data)  # Reset state and time.
while data.time < duration:
  mujoco.mj_step(model, data)
  if len(frames) < data.time * framerate:
    renderer.update_scene(data)
    pixels = renderer.render()
    frames.append(pixels)
media.show_video(frames, fps=framerate)

Hmmm, the video is playing, but nothing is moving, why is that?

This is because this model has no [degrees of freedom](https://www.google.com/url?sa=D&q=https%3A%2F%2Fen.wikipedia.org%2Fwiki%2FDegrees_of_freedom_(mechanics)) (DoFs). The things that move (and which have inertia) are called *bodies*.

We add DoFs by adding *joints* to bodies, specifying how they can move with respect to their parents, a [freejoint](https://mujoco.readthedocs.io/en/stable/XMLreference.html#body-freejoint) will give our turtle the freedom she desires.

# Free Turtley!

Let's go back to our turtle and give it freedom to move:

> "oh is it going to roam around on its own?"

no not yet, but that's a great idea!

> "can we at least make it look a bit more like a turtle?"

Okay, lets tweak a little bit...

*time passes* ...

*more time passes* ...

In [ ]:
n_frames = 200
height = 240
width = 320
frames = []
renderer = mujoco.Renderer(model, height, width)

free_body_MJCF = """
<mujoco>
  <asset>
    <texture name="grid" type="2d" builtin="checker" rgb1=".1 .2 .3"
    rgb2=".2 .3 .4" width="300" height="300" mark="edge" markrgb=".2 .3 .4"/>
    <material name="grid" texture="grid" texrepeat="2 2" texuniform="true"
    reflectance=".2"/>
  </asset>

  <worldbody>
    <light pos="0 0 1" mode="trackcom"/>
    <geom name="ground" type="plane" pos="0 0 -.5" size="2 2 .1" material="grid" solimp=".99 .99 .01" solref=".001 1"/>
    <body name="box_and_sphere" pos="0 0 0">
      <freejoint/>
      <geom name="body" type="ellipsoid" size=".2 .1 .1" rgba="0 1 0.5 1"/>
      <geom name="shell" type="ellipsoid" pos="-0.06 .0 .02" size=".18 .14 .14" rgba="0.5 1 0 1"/>
      <camera name="fixed" pos="0 -.6 .3" xyaxes="1 0 0 0 1 2"/>
      <camera name="track" pos="0 -.6 .3" xyaxes="1 0 0 0 1 2" mode="track"/>
    </body>
  </worldbody>
</mujoco>
"""
model = mujoco.MjModel.from_xml_string(free_body_MJCF)
renderer = mujoco.Renderer(model, 400, 600)
data = mujoco.MjData(model)
mujoco.mj_forward(model, data)
renderer.update_scene(data, "fixed")

for i in range(n_frames):
  while data.time < i/120.0: #1/4x real time
    mujoco.mj_step(model, data)
  renderer.update_scene(data, "track")
  frame = renderer.render()
  frames.append(frame)
media.show_video(frames, fps=framerate)


Let's test rendering this body rolling on the floor, in slow-motion, with a random velocity while visualizing contact points and forces.

> Why?

Because we can.

In [ ]:
n_frames = 200
height = 240
width = 320
frames = []
renderer = mujoco.Renderer(model, height, width)

# visualize contact frames and forces, make body transparent
options = mujoco.MjvOption()
mujoco.mjv_defaultOption(options)
options.flags[mujoco.mjtVisFlag.mjVIS_CONTACTPOINT] = True
options.flags[mujoco.mjtVisFlag.mjVIS_CONTACTFORCE] = True
options.flags[mujoco.mjtVisFlag.mjVIS_TRANSPARENT] = True

# tweak scales of contact visualization elements
model.vis.scale.contactwidth = 0.1
model.vis.scale.contactheight = 0.03
model.vis.scale.forcewidth = 0.05
model.vis.map.force = 0.3

# random initial rotational velocity:
mujoco.mj_resetData(model, data)
data.qvel[3:6] = 5*np.random.randn(3)

# simulate and render
for i in range(n_frames):
  while data.time < i/120.0: #1/4x real time
    mujoco.mj_step(model, data)
  renderer.update_scene(data, "track", options)
  frame = renderer.render()
  frames.append(frame)

# show video
media.show_video(frames, fps=30)

Okay now let's give it some fins for stability.

> Yay

> ... wait is this a turtle or a tortoise?

Umm, let's call it a turtle for now, if you're curious, check out [Turtle vs tortoise: what's the difference?](https://www.discoverwildlife.com/animal-facts/reptiles/turtle-vs-tortoise)

For now, back to the finny-leg-things, adding ellipsoids and using `euler` to rotate them.

In [ ]:

free_body_MJCF = """
<mujoco>
  <asset>
    <texture name="grid" type="2d" builtin="checker" rgb1=".1 .2 .3"
    rgb2=".2 .3 .4" width="300" height="300" mark="edge" markrgb=".2 .3 .4"/>
    <material name="grid" texture="grid" texrepeat="2 2" texuniform="true"
    reflectance=".2"/>
  </asset>

  <worldbody>
    <light pos="0 0 1" mode="trackcom"/>
    <geom name="ground" type="plane" pos="0 0 -.5" size="2 2 .1" material="grid" solimp=".99 .99 .01" solref=".001 1"/>
    <body name="box_and_sphere" pos="0 0 0">
      <freejoint/>
      <geom name="body" type="ellipsoid" size=".2 .1 .1" rgba="0 1 0.5 1"/>
      <geom name="shell" type="ellipsoid" pos="-0.06 .0 .02" size=".18 .14 .14" rgba="0.5 1 0 1"/>
      <geom name="flip1" type="ellipsoid" pos="0.04 -0.07 -0.05" euler="45 0 0" size=".05 .15 .05" rgba="0 1 0.5 1"/>
      <geom name="flip2" type="ellipsoid" pos="0.04 0.07 -0.05" euler="-45 0 0" size=".05 .15 .05" rgba="0 1 0.5 1"/>
      <geom name="flip3" type="ellipsoid" pos="-0.17 -0.05 -0.05" euler="45 0 0" size=".05 .15 .05" rgba="0 1 0.5 1"/>
      <geom name="flip4" type="ellipsoid" pos="-0.17 0.05 -0.05" euler="-45 0 0" size=".05 .15 .05" rgba="0 1 0.5 1"/>



      <camera name="fixed" pos="0 -.6 .3" xyaxes="1 0 0 0 1 2"/>
      <camera name="track" pos="0 -.6 .3" xyaxes="1 0 0 0 1 2" mode="track"/>
    </body>
  </worldbody>
</mujoco>
"""
model = mujoco.MjModel.from_xml_string(free_body_MJCF)
renderer = mujoco.Renderer(model, 400, 600)
data = mujoco.MjData(model)
mujoco.mj_forward(model, data)

duration = 3.8  # (seconds)
framerate = 60  # (Hz)

frames = []
mujoco.mj_resetData(model, data)
while data.time < duration:
  mujoco.mj_step(model, data)
  if len(frames) < data.time * framerate:
    renderer.update_scene(data)
    pixels = renderer.render()
    frames.append(pixels)

# Simulate and display video.
media.show_video(frames, fps=framerate)

Okay, yes, I cheated that wasn't the same simulation, but it sure was pretty.

If you're interested (because I know you are), here it is with a random throw.

In [ ]:
n_frames = 500
height = 240
width = 320
frames = []
renderer = mujoco.Renderer(model, height, width)

# visualize contact frames and forces, make body transparent
options = mujoco.MjvOption()
mujoco.mjv_defaultOption(options)
options.flags[mujoco.mjtVisFlag.mjVIS_CONTACTPOINT] = True
options.flags[mujoco.mjtVisFlag.mjVIS_CONTACTFORCE] = True
options.flags[mujoco.mjtVisFlag.mjVIS_TRANSPARENT] = True

# tweak scales of contact visualization elements
model.vis.scale.contactwidth = 0.1
model.vis.scale.contactheight = 0.03
model.vis.scale.forcewidth = 0.05
model.vis.map.force = 0.3

# random initial rotational velocity:
mujoco.mj_resetData(model, data)
data.qvel[3:6] = 5*np.random.randn(3)

# simulate and render
for i in range(n_frames):
  while data.time < i/120.0: #1/4x real time
    mujoco.mj_step(model, data)
  renderer.update_scene(data, "track", options)
  frame = renderer.render()
  frames.append(frame)

# show video
media.show_video(frames, fps=30)

# Let's get moving

We need to get this thing to move now.

To do this we'll try start with a differential drive, following the pattern of Pranav Bhounsule's tutorial here: https://www.youtube.com/watch?v=I5QvXfo8L4o

We connect wheels to the body with joints, and then attach an actuator to be the little motors.

In [ ]:
car = """
<mujoco>
	<option gravity="0 0 -9.81" />
	<worldbody>
        <light diffuse=".5 .5 .5" pos="0 0 3" dir="0 0 -1"/>
		<geom type="plane" size="5 5 0.1" rgba=".9 .9 .9 1"/>
		<body name = "chassis" pos="0 0 0.2" euler='0 90 0'>
			<joint type="free"/>
			<geom type="box" size=".05 .2 .5" rgba=".9 .9 0 1"/>
			<site name="marker" pos = "0 0 0.1" size="0.1" />
			<body name="left-tire" pos="0 0.3 -0.5" euler='90 0 0'>
				<joint name = "left-wheel" type="hinge" axis="0 0 -1"/>
				<geom type="cylinder" size=".2 0.05" rgba="0 .9 0 1"/>
			</body>
			<body name="right-tire" pos="0 -0.3 -0.5" euler='90 0 0'>
				<joint name = "right-wheel" type="hinge" axis="0 0 -1"/>
				<geom type="cylinder" size=".2 0.05" rgba="0 .9 0 1"/>
			</body>
		</body>

    <camera name="fixed" pos="0 -10 4" xyaxes="1 0 0 0 1 2"/>

	</worldbody>
	<sensor>
		<framepos objtype="site" objname="marker"/>
	</sensor>
	<actuator>
		<velocity name="left-velocity-servo" joint="left-wheel" kv="100"/>
		<velocity name="right-velocity-servo" joint="right-wheel" kv="100"/>
	</actuator>
</mujoco>

"""

model = mujoco.MjModel.from_xml_string(car)

That's the simple diff drive car model

In [ ]:
renderer = mujoco.Renderer(model, 400, 600)
data = mujoco.MjData(model)
mujoco.mj_forward(model, data)

framerate = 60  # (Hz)

frames = []
mujoco.mj_resetData(model, data)

renderer.update_scene(data, camera="fixed")

def step_to_next_frame(i, start_time_offset):
  loops=0
  while (data.time - start_time_offset) < i/framerate and loops < 100:
    #print(data.time)
    loops = loops + 1
    #print(loops)
    mujoco.mj_step(model, data)

    #times.append(data.time)
    #sensordata.append(data.sensor('accelerometer').data.copy())

# simulate and render
def render_frames(n):
  start_time = data.time * framerate
  for i in range(n):
    step_to_next_frame(i, start_time)
    renderer.update_scene(data, camera="fixed")
    frame = renderer.render()
    frames.append(frame)

data.ctrl[0] = 10;
data.ctrl[1] = 10;

render_frames(60)

data.ctrl[0] = 6;
data.ctrl[1] = -6;

render_frames(60)

data.ctrl[0] = 1;
data.ctrl[1] = 1;

render_frames(60)

data.ctrl[0] = 6;
data.ctrl[1] = -6;

render_frames(30)

data.ctrl[0] = 1;
data.ctrl[1] = 1;

render_frames(60)

data.ctrl[0] = 6;
data.ctrl[1] = -6;

render_frames(30)

# display video.
media.show_video(frames, fps=framerate)


# Turtle gets going

Now we need to figure out a way to mash up these two models (without it breaking, which it has done, a lot).

In [ ]:
car = """
<mujoco>
	<option gravity="0 0 -9.81" />
	<worldbody>
    <light diffuse=".5 .5 .5" pos="0 0 3" dir="0 0 -1"/>
		<geom type="plane" size="5 5 0.1" rgba=".9 .9 .9 1"/>
		<body name = "chassis" pos="0 0 1.2" euler='0 90 0'>
			<joint type="free"/>
 			<geom type="box" size=".2 0.2 0.5" rgba=".9 .9 0 1"/>
			<site name="marker" pos = "0 0 0.1" size="0.1" />
			<body name="left-tire" pos="0 0.5 -0.5" euler='90 0 0'>
				<joint name = "left-wheel" type="hinge" axis="0 0 -1"/>
				<geom type="cylinder" size=".2 0.05" rgba="0 .9 0 1"/>
			</body>
			<body name="right-tire" pos="0 -0.5 -0.5" euler='90 0 0'>
				<joint name = "right-wheel" type="hinge" axis="0 0 -1"/>
				<geom type="cylinder" size=".2 0.05" rgba="0 .9 0 1"/>
			</body>
		</body>

    <camera name="fixed" pos="0 -10 4" xyaxes="1 0 0 0 1 2"/>

	</worldbody>
	<sensor>
		<framepos objtype="site" objname="marker"/>
	</sensor>
	<actuator>
		<velocity name="left-velocity-servo" joint="left-wheel" kv="100"/>
		<velocity name="right-velocity-servo" joint="right-wheel" kv="100"/>
	</actuator>
</mujoco>

"""

turtle = """
<mujoco>
  <option gravity="0 0 -9.81" />
  <asset>
    <texture name="grid" type="2d" builtin="checker" rgb1=".1 .2 .3"
    rgb2=".2 .3 .4" width="300" height="300" mark="edge" markrgb=".2 .3 .4"/>
    <material name="grid" texture="grid" texrepeat="2 2" texuniform="true"
    reflectance=".2"/>
  </asset>

  <worldbody>

    <light diffuse=".5 .5 .5" pos="0 0 3" dir="0 0 -1"/>
    <geom name="ground" type="plane" pos="0 0 -.5" size="100 100 .1" material="grid" solimp=".99 .99 .01" solref=".001 1"/>
    <body name="turtle" pos="0 0 0">
      <freejoint/>
      <geom name="body" type="ellipsoid" size=".4 .2 .2" rgba="0 1 0.5 1"/>
      <geom name="shell" type="ellipsoid" pos="-0.12 .0 .04" size=".36 .28 .28" rgba="0.5 1 0 1"/>
      <body name="left-tire" pos="0 0.5 -.15"  euler='90 0 0'>
				<joint name = "left-wheel" type="hinge" axis="0 0 -1"/>
				<geom type="cylinder" size=".2 0.05" rgba=".4 .9 0 0.3"/>
			</body>
      <body name="right-tire" pos="0 -0.5 -.15"  euler='90 0 0'>
				<joint name = "right-wheel" type="hinge" axis="0 0 -1"/>
				<geom type="cylinder" size=".2 0.05" rgba=".4 .9 0 0.3"/>
			</body>
      <body name="right-flipper" pos="0.08 -.14 -0.1" euler="45 0 0" >
				<joint name = "right-axis" type="hinge" axis="0 1 0.2"/>
        <geom name="flip1" type="ellipsoid" size=".1 .3 .1" rgba="0 1 0.5 1"/>
      </body>
      <geom name="flip2" type="ellipsoid" pos="0.08 0.14 -0.1" euler="-45 0 0" size=".1 .3 .1" rgba="0 1 0.5 1"/>
      <geom name="flip3" type="ellipsoid" pos="-0.34 -0.14 -0.1" euler="45 0 0" size=".1 .3 .1" rgba="0 1 0.5 1"/>
      <geom name="flip4" type="ellipsoid" pos="-0.34 0.14 -0.1" euler="-45 0 0" size=".1 .3 .1" rgba="0 1 0.5 1"/>

      <camera name="fixed" pos="0 -4 2" xyaxes="1 0 0 0 1 2"/>
      <camera name="track" pos="0 -.6 .3" xyaxes="1 0 0 0 1 2" mode="track"/>

    </body>


  </worldbody>
  <actuator>
		<velocity name="left-velocity-servo" joint="left-wheel" kv="100"/>
		<velocity name="right-velocity-servo" joint="right-wheel" kv="100"/>
	</actuator>
</mujoco>
"""

model = mujoco.MjModel.from_xml_string(turtle)


In [ ]:
renderer = mujoco.Renderer(model, 400, 600)
data = mujoco.MjData(model)
mujoco.mj_forward(model, data)

framerate = 60  # (Hz)

frames = []
mujoco.mj_resetData(model, data)

scene_option = mujoco.MjvOption()
scene_option.flags[mujoco.mjtVisFlag.mjVIS_JOINT] = True

renderer.update_scene(data, camera="fixed")

def step_to_next_frame(i, start_time_offset):
  loops=0
  while (data.time - start_time_offset) < i/framerate and loops < 100:
    #print(data.time)
    loops = loops + 1
    #print(loops)
    mujoco.mj_step(model, data)

    #times.append(data.time)
    #sensordata.append(data.sensor('accelerometer').data.copy())

# simulate and render
def render_frames(n):
  start_time = data.time * framerate
  for i in range(n):
    step_to_next_frame(i, start_time)
    renderer.update_scene(data, scene_option=scene_option, camera="fixed")
    frame = renderer.render()
    frames.append(frame)

for i in np.random.rand(15,2):
  data.ctrl[0] = 3*i[0]+1;
  data.ctrl[1] = 2*i[1]+1;
  render_frames(60)

# display video.
media.show_video(frames, fps=framerate)

... and we have a free roaming turtle!

# 🥳 🐢

# What's next?

*  implement `turtle.forward(cells)` measuring distance to go a defined distance forward.
*  implement `turtle.left(degrees)` and `turtle.right(degrees)` measuring the change in angle to decide when to stop rotating
* create a module and pip package to make it easy to `pip install` and `import` the `turtle-sim` module and have it behave similarly to `turtle`.